<h1>Time Series Forecasting Recurrent Neural Network (RNN) </h1>

This notebook aims to use deep learning to forecast the time series the "CLOSE" of the SNP 500 stock using the stock data from other industries. 

Libraries of interest: <b>
    
    TensorFlow
    Keras
    
</b>

In [1]:
# Download the following libraries if you do not have them:
# !pip install --upgrade tensorflow

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Keras imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [4]:
df = pd.read_csv("data\output.csv")
df.head()

,Date,Open_Pharm,High_Pharm,Low_Pharm,Close_Pharm,Adj Close_Pharm,Volume_Pharm,Open_Semicon,High_Semicon,Low_Semicon,...,Low_Utilities,Close_Utilities,Adj Close_Utilities,Volume_Utilities,Open_Consumer,High_Consumer,Low_Consumer,Close_Consumer,Adj Close_Consumer,Volume_Consumer
0,4/1/2010,33.150002,33.544998,33.020000,33.535000,26.125103,1622800,28.350000,28.660000,28.299999,...,31.010000,31.080000,21.588554,8217600,29.900000,30.110001,29.900000,30.000000,25.967613,5443900
1,5/1/2010,33.580002,33.580002,33.134998,33.220001,25.879725,1933200,28.450001,28.540001,28.080000,...,30.559999,30.709999,21.331547,18023700,30.010000,30.139999,29.820000,30.110001,26.062828,6162200
2,6/1/2010,33.250000,33.384998,33.169998,33.384998,26.008247,1492400,28.209999,28.450001,28.080000,...,30.690001,30.889999,21.456575,12745100,30.090000,30.219999,30.020000,30.150000,26.097452,4246900
3,7/1/2010,33.285000,33.419998,33.230000,33.330002,25.965412,764600,28.090000,28.160000,27.760000,...,30.639999,30.750000,21.359331,6563100,30.340000,30.410000,30.139999,30.400000,26.313848,5736700
4,8/1/2010,33.259998,33.369999,33.185001,33.340000,25.973190,1218200,27.940001,28.549999,27.870001,...,30.520000,30.719999,21.338493,6267600,30.290001,30.410000,30.120001,30.389999,26.305195,6438000


In [5]:
df.values.shape

(2571, 55)

Now we need to pinpoint the variable of interest as well as the shifting rate, which is the amount of days into the future we want to predict.

In [6]:
target = 'Adj Close_SNP500'

In [7]:
shift_days = 1
shift_steps = 1 * 365 # Number of years

This is the new dataframe with the time-shifted data

In [8]:
df_target = df[target].shift(-shift_steps)

<h4>Ensure that the df has been successfully shifted:

In [9]:
df[target].head(shift_steps + 5)

0      1132.989990
1      1136.520020
2      1137.140015
3      1141.689941
4      1144.979980
          ...     
365    1265.420044
366    1267.640015
367    1271.500000
368    1278.359985
369    1295.520020
Name: Adj Close_SNP500, Length: 370, dtype: float64

In [10]:
df_target.head(5)

0    1265.420044
1    1267.640015
2    1271.500000
3    1278.359985
4    1295.520020
Name: Adj Close_SNP500, dtype: float64